In [1]:
# requirements 
from __future__ import unicode_literals, print_function, division
from io import open 
import unicodedata
import re 
import random 
import math

import torch 
import torch.nn as nn 
from torch import optim 
import torch.nn.functional as F 

import numpy as np 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Create a unique index per word to use as inputs and targets of the networks later.

In [2]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return "".join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn"
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", " \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [3]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s-cleaned.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [5]:
# Only include sentences with max 10 words for quick testing.
# Increase for more complexity
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p): 
    return len(p[0].split(" ")) < MAX_LENGTH and \
        len(p[1].split(" ")) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)
    
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
# Remove CC lines
# Initialize an empty list to store the processed lines
def removeCC(filepath, save_name):
    processed_lines = []

    # Open the file and read line by line
    with open(filepath, "r", encoding="utf-8") as file:
        for line in file:
            # Find the index of "CC-BY"
            index = line.find("CC-BY")
            
            # If "CC-BY" is found in the line
            if index != -1:
                # Remove "CC-BY" and everything after it and add the processed line to the list
                processed_lines.append(line[:index].strip())
            else:
                # If "CC-BY" is not found, add the original line to the list
                processed_lines.append(line.strip())

    # Join the processed lines into a single string
    processed_text = "\n".join(processed_lines)

    # Write the processed text to a new file
    with open(f"./data/{save_name}", "w", encoding="utf-8") as file:
        file.write(processed_text)

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('en', 'it', True)
print(random.choice(pairs))

Reading lines...
Read 374932 sentence pairs
Trimmed to 34220 sentence pairs
Counting words...
Counted words:
it 5670
en 3340
['sono sicura che tom sia molto orgoglioso di te', 'i m sure tom is very proud of you']


In [98]:
class EncoderTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1, num_layers=6, nhead=8):
        super(EncoderTransformer, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Dropout(dropout_p),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.Dropout(dropout_p),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.Dropout(dropout_p),
            nn.ReLU(),
        )
        encoder_layer = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        embedded = self.pos_encoder(embedded)
        output = self.transformer_encoder(embedded)
        return output


In [99]:
class DecoderTransformer(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, num_layers=6, nhead=8):
        super(DecoderTransformer, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.pos_encoder = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Dropout(dropout_p),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.Dropout(dropout_p),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.Dropout(dropout_p),
            nn.ReLU(),
        )
        decoder_layer = nn.TransformerDecoderLayer(hidden_size, nhead)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, encoder_outputs, src_mask=None, tgt_mask=None, memory_mask=None):
        input = self.dropout(input)
        input = self.pos_encoder(input)
        output = self.transformer_decoder(input, encoder_outputs, tgt_mask=tgt_mask, memory_mask=memory_mask)
        output = self.out(output)
        return output


In [100]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('en', 'it', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [101]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [102]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [103]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [104]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [105]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [106]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [107]:
input_lang.n_words, output_lang.n_words

(5670, 3340)

In [108]:
input_vocab_size = input_lang.n_words
target_vocab_size = output_lang.n_words
batch_size = 32
hidden_size = 128  # or any other value you prefer

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderTransformer(input_lang.n_words, hidden_size).to(device)
decoder = DecoderTransformer(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 30, print_every=1, plot_every=20)

Reading lines...
Read 374932 sentence pairs
Trimmed to 34220 sentence pairs
Counting words...
Counted words:
it 5670
en 3340


AssertionError: For batched (3-D) `query`, expected `key` and `value` to be 3-D but found 2-D and 2-D tensors respectively

In [ ]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> sono molto felice per voi
= i m very happy for you
< i m very happy for you <EOS>

> sono una persona razionale
= i m a rational person
< i m a rational person <EOS>

> tu sei troppo sospettosa di tutto
= you re too suspicious about everything
< you re too suspicious about everything <EOS>

> e dura
= you re harsh
< you re harsh <EOS>

> lei e qui
= you re here
< she is here <EOS>

> io sono molto grato per il vostro aiuto
= i m very grateful for your help
< i m very grateful for your help <EOS>

> sono un fumettista
= i m a cartoonist
< i m a cartoonist person <EOS>

> sono fuori controllo
= i m out of control
< i m out of control <EOS>

> tu sei davvero brava
= you re really good
< you re really good at good <EOS>

> sta facendo una passeggiata
= he is taking a walk
< he is making a walk <EOS>



In [ ]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])


evaluateAndShowAttention("mi piace")

input = mi piace
output = i m glad i hired children <EOS>


C:\Users\Leo\AppData\Local\Temp\ipykernel_7852\2456228459.py:8: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
C:\Users\Leo\AppData\Local\Temp\ipykernel_7852\2456228459.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)
C:\Users\Leo\AppData\Local\Temp\ipykernel_7852\2456228459.py:16: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
